<a href="https://colab.research.google.com/github/Sandeepkumaramgothu/ASE_practice/blob/main/Spect_2_Gun_Other.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install pydub
from pydub import AudioSegment

In [3]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random



In [4]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report, confusion_matrix
from pydub import AudioSegment


In [22]:
from pydub import AudioSegment
import os

def split_audio_file(input_file, output_directory, clip_duration=1000):
    # Determine the format from the file extension
    file_format = input_file.split('.')[-1]

    # Load the audio file based on its format
    if file_format == 'mp3':
        audio = AudioSegment.from_mp3(input_file)
    elif file_format == 'wav':
        audio = AudioSegment.from_wav(input_file)
    else:
        raise ValueError(f"Unsupported file format: {file_format}")

    # Clip duration in milliseconds
    clip_duration_ms = clip_duration

    # Split the audio and save clips
    for i, start in enumerate(range(0, len(audio), clip_duration_ms)):
        clip = audio[start:start + clip_duration_ms]
        output_file = os.path.join(output_directory, f"{os.path.splitext(os.path.basename(input_file))[0]}_{i}.{file_format}")
        clip.export(output_file, format=file_format)
        print(f"Saved {output_file}")

def process_directory(source_directory, target_directory):
    # Create target directory structure and split files
    for root, dirs, files in os.walk(source_directory):
        print(f"\nProcessing directory: {root} \n")
        for category in dirs:
            category_dir = os.path.join(target_directory, category)
            os.makedirs(category_dir, exist_ok=True)
        for file in files:
            if file.endswith(('.wav', '.mp3')):
                input_file_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, source_directory)
                output_directory_path = os.path.join(target_directory, relative_path)
                os.makedirs(output_directory_path, exist_ok=True)
                split_audio_file(input_file_path, output_directory_path)

# Base directory for datasets
base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
sound_dir = os.path.join(base_dir, 'Sound')
sound_1_sec_dir = os.path.join(base_dir, 'Sound_1_Sec')

# Process each main category within the sound directory
for category in ['Train', 'Test']:
    source_directory = os.path.join(sound_dir, category)
    target_directory = os.path.join(sound_1_sec_dir, category)
    process_directory(source_directory, target_directory)


Processing directory: /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train 

Processing directory: /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/M16 

Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (29)_0.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (29)_1.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (32)_0.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (32)_1.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (25)_0.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (25)_1.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (100)_0.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (100)_1.wav
Saved /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M16/5 (31)_0.wav
Saved /content

In [25]:

def create_mel_spectrogram(file_path, save_path, n_mels=128, n_fft=2048, hop_length=512, sr=16000):
    """ This function generates a mel spectrogram from a sound file and saves it as an image. """
    y, sr = librosa.load(file_path, sr=sr)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram')
    plt.tight_layout()

    plt.savefig(save_path)
    #plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    print(f'Mel spectrogram saved as {save_path}\n')
    plt.close()

def prepare_dataset(sound_dir, image_dir):
    """ Processes each category in train and test directories to create spectrogram images. """
    for set_type in ['Train', 'Test']:
        set_sound_dir = os.path.join(sound_dir, set_type)
        set_image_dir = os.path.join(image_dir, set_type)
        categories = os.listdir(set_sound_dir)
        print(f'Processing {set_type} set with {len(categories)} categories\n')
        for category in categories:
            filecounter=0
            print(f'Processing {category} category... in {set_type} ')
            cat_sound_dir = os.path.join(set_sound_dir, category)
            cat_image_dir = os.path.join(set_image_dir, category)
            os.makedirs(cat_image_dir, exist_ok=True)
            sound_files = [f for f in os.listdir(cat_sound_dir) if f.endswith('.wav') or f.endswith('.mp3')]
            for sound_file in sound_files:
                file_path = os.path.join(cat_sound_dir, sound_file)
                print(f'Processing {file_path}')
                #Change is here to handle both .wav and .mp3 files
                image_path = os.path.join(cat_image_dir, os.path.splitext(sound_file)[0] + '.png')
                create_mel_spectrogram(file_path, image_path)
                filecounter+=1
            print(f"Processed {category} with {filecounter} files \n\n")


In [26]:
base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
sound_dir = os.path.join(base_dir, 'Sound_1_Sec')
image_dir = os.path.join(base_dir, 'Images_28')

prepare_dataset(sound_dir, image_dir)

Streaming output truncated to the last 5000 lines.

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M249/6 (66)_1.wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/M249/6 (66)_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M249/6 (81)_0.wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/M249/6 (81)_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M249/6 (81)_1.wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/M249/6 (81)_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M249/6 (73)_0.wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/M249/6 (73)_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/M249/6 (73)_1.wav
Mel spectrogram saved as /conten

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=384
  warnings.warn(


Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-large-129051_18.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-6801 (1)_0.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-6801 (1)_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-6801 (1)_1.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-6801 (1)_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-6801 (1)_2.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-6801 (1)_2.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-6801 (1)_3.mp3
Mel spectrogram saved as /content/drive

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1536
  warnings.warn(


Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-6801 (1)_6.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-5981 (1)_0.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-5981 (1)_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-5981 (1)_1.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-5981 (1)_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-5981 (1)_2.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Train/Explosion/explosion-5981 (1)_2.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Train/Explosion/explosion-5981 (1)_3.mp3


...

Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/small-explosion-94980_2.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/small-explosion-129477 (1)_0.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/small-explosion-129477 (1)_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/small-explosion-129477 (1)_1.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/small-explosion-129477 (1)_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/small-explosion-103931_0.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/small-explosion-103931_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/small-explosion-103931_1.mp3
Mel s

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=768
  warnings.warn(


Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/mrm_gas_steam_escape-76024_24.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/medium-explosion-40472 (1)_0.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/medium-explosion-40472 (1)_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/medium-explosion-40472 (1)_1.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/medium-explosion-40472 (1)_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/medium-explosion-40472 (1)_2.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/medium-explosion-40472 (1)_2.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/medium-explosion-404

/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=320
  warnings.warn(


Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/metal-design-explosion-13491_8.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/large-underwater-explosion-190270_0.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/large-underwater-explosion-190270_0.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/large-underwater-explosion-190270_1.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/large-underwater-explosion-190270_1.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound_1_Sec/Test/Explosion/large-underwater-explosion-190270_2.mp3
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_28/Test/Explosion/large-underwater-explosion-190270_2.png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Soun

In [6]:
base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
sound_dir = os.path.join(base_dir, 'Sound_1_Sec')
image_dir = os.path.join(base_dir, 'Images_28')


In [ ]:
num_classes = 10

n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

height = 128
width = 1 + int((audio_length_samples - n_fft) / hop_length)
#width = 256
channels = 3

batch_size = 32

print(f"height: {height}")
print(f"width: {width}")
print(f"channels: {channels}")


train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()



train_generator = train_datagen.flow_from_directory(
        os.path.join(image_dir, 'Train'),
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        os.path.join(image_dir, 'Test'),
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

model = tf.keras.Sequential()

# Convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten layer to feed into fully connected layers
model.add(tf.keras.layers.Flatten())

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy','precision','recall','f1_score'])

epochs = 200
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_generator.reset()
    model.fit(train_generator,steps_per_epoch=len(train_generator),epochs=1,verbose=1)


loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

model.save('el-amadi_cnn_standardized_unknown.keras')

height: 128
width: 28
channels: 3
Found 2359 images belonging to 10 classes.
Found 474 images belonging to 10 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


74/74 ━━━━━━━━━━━━━━━━━━━━ 41s 460ms/step - accuracy: 0.3102 - f1_score: 0.1050 - loss: 9.3891 - precision: 0.3166 - recall: 0.1217
Epoch 2/200
74/74 ━━━━━━━━━━━━━━━━━━━━ 31s 414ms/step - accuracy: 0.4815 - f1_score: 0.0968 - loss: 1.7640 - precision: 0.7878 - recall: 0.2464
Epoch 3/200
74/74 ━━━━━━━━━━━━━━━━━━━━ 30s 402ms/step - accuracy: 0.4557 - f1_score: 0.0915 - loss: 1.6608 - precision: 0.8663 - recall: 0.2829
Epoch 4/200
74/74 ━━━━━━━━━━━━━━━━━━━━ 31s 420ms/step - accuracy: 0.4627 - f1_score: 0.1052 - loss: 1.6833 - precision: 0.8276 - recall: 0.3020
Epoch 5/200
74/74 ━━━━━━━━━━━━━━━━━━━━ 30s 406ms/step - accuracy: 0.4745 - f1_score: 0.1412 - loss: 1.6092 - precision: 0.8713 - recall: 0.3198
Epoch 6/200
74/74 ━━━━━━━━━━━━━━━━━━━━ 31s 420ms/step - accuracy: 0.4875 - f1_score: 0.1553 - loss: 1.5097 - precision: 0.8730 - recall: 0.3584
Epoch 7/200
74/74 ━━━━━━━━━━━━━━━━━━━━ 30s 404ms/step - accuracy: 0.4931 - f1_score: 0.1687 - loss: 1.5033 - precision: 0.8750 - recall: 0.3569
Epoc

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 9
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 2

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)
#width = 256

channels = 3

batch_size = 32

model = tf.keras.models.load_model('/content/el-amadi_cnn_standardized_unknown.keras')


test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        os.path.join(image_dir, 'Test'),
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
#print("Test Precision:", precision)
#print("Test Recall:", recall)
#print("Test F1 Score:", f1_score)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

print("predicted values : ",predicted_classes)
print("True values      : ",true_classes)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)




print("\nTesting New matrices")

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 216 images belonging to 9 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 368ms/step - accuracy: 0.5121 - loss: 3.1825
Test Loss: 3.394925355911255
Test Accuracy: 0.49074074625968933
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 371ms/step
predicted values :  [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 3 4 1 1 1 0 1 1 1 1 5 1
 1 6 6 1 7 1 1 6 1 4 6 7 2 7 7 7 5 3 2 7 7 0 4 3 4 7 4 2 4 4 4 4 2 2 2 4 7
 0 1 6 6 6 1 5 7 7 0 7 6 6 7 5 7 2 7 6 7 5 6 4 5 4 4 1 4 4 5 4 4 7 4 4 4 4
 4 4 4 4 5 1 4 4 1 4 7 0 1 6 6 6 1 5 7 7 0 7 6 6 7 5 7 2 7 6 7 5 6 2 5 3 6
 3 6 6 4 3 4 4 2 4 7 6 6 6 3 4 4 3 4 2 6 7 3 7 7 5 7 7 7 0 7 7 2 7 7 7 3 7
 4 7 2 7 3 5 7 8 8 8 8 8 8 8 8 6 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
True values      :  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6
 6 6

In [8]:
#This VERSION is of GPT

#This VERSION is of GPT

In [9]:

def create_mel_spectrogram(file_path, save_path, n_mels=128, n_fft=2048, hop_length=512, sr=22050):
    """ This function generates a mel spectrogram from a sound file and saves it as an image. """
    y, sr = librosa.load(file_path, sr=sr)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    S_DB = librosa.amplitude_to_db(S, ref=np.max)

    plt.figure(figsize=(10, 4))
    librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Mel spectrogram')
    plt.tight_layout()

    plt.savefig(save_path)
    #plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    print(f'Mel spectrogram saved as {save_path}\n')
    plt.close()

def prepare_dataset(sound_dir, image_dir):
    """ Processes each category in train and test directories to create spectrogram images. """
    for set_type in ['Train', 'Test']:
        set_sound_dir = os.path.join(sound_dir, set_type)
        set_image_dir = os.path.join(image_dir, set_type)
        categories = os.listdir(set_sound_dir)
        print(f'Processing {set_type} set with {len(categories)} categories')
        for category in categories:
            filecounter=0
            print(f'Processing {category} category... in {set_type} ')
            cat_sound_dir = os.path.join(set_sound_dir, category)
            cat_image_dir = os.path.join(set_image_dir, category)
            os.makedirs(cat_image_dir, exist_ok=True)
            sound_files = [f for f in os.listdir(cat_sound_dir) if f.endswith('.wav') or f.endswith('.mp3')]
            for sound_file in sound_files:
                file_path = os.path.join(cat_sound_dir, sound_file)
                print(f'Processing {file_path}')
                image_path = os.path.join(cat_image_dir, sound_file.replace('.wav', '.png'))
                create_mel_spectrogram(file_path, image_path)
                filecounter+=1
            print(f"Processed {category} with {filecounter} files \n\n")




In [10]:
base_dir = '/content/drive/MyDrive/DataSet/Dataset_Version_2'
sound_dir = os.path.join(base_dir, 'Sound')
image_dir = os.path.join(base_dir, 'Images_256')

prepare_dataset(sound_dir, image_dir)

Processing Train set with 9 categories
Processing M16 category... in Train 
Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/M16/5 (29).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_256/Train/M16/5 (29).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/M16/5 (32).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_256/Train/M16/5 (32).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/M16/5 (25).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_256/Train/M16/5 (25).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/M16/5 (100).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dataset_Version_2/Images_256/Train/M16/5 (100).png

Processing /content/drive/MyDrive/DataSet/Dataset_Version_2/Sound/Train/M16/5 (31).wav
Mel spectrogram saved as /content/drive/MyDrive/DataSet/Dat

In [11]:
num_classes = 9

n_fft = 2048
hop_length = 512
sr = 22050
T = 2

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

height = 128
#width = 1 + int((audio_length_samples - n_fft) / hop_length)
width = 256
channels = 3

batch_size = 32

print(f"height: {height}")
print(f"width: {width}")
print(f"channels: {channels}")


train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    rescale=1./255,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


height: 128
width: 256
channels: 3


In [12]:
#train_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    os.path.join(image_dir, 'Train'),
    target_size=(height, width),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    os.path.join(image_dir, 'Test'),
    target_size=(height, width),
    batch_size=32,
    class_mode='categorical',
    shuffle=False

)



Found 635 images belonging to 9 classes.
Found 216 images belonging to 9 classes.


In [16]:


model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 256, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(9, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','precision','recall','f1_score'])

model.fit(train_generator, epochs=200, validation_data=test_generator)




/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 924ms/step - accuracy: 0.1076 - f1_score: 0.0868 - loss: 2.4375 - precision: 0.1451 - recall: 0.0072 - val_accuracy: 0.1204 - val_f1_score: 0.0399 - val_loss: 2.1913 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 807ms/step - accuracy: 0.1462 - f1_score: 0.1037 - loss: 2.1849 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.1157 - val_f1_score: 0.0342 - val_loss: 2.1540 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 741ms/step - accuracy: 0.1638 - f1_score: 0.1285 - loss: 2.1236 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.2917 - val_f1_score: 0.1998 - val_loss: 2.0933 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 734ms/step - accuracy: 0.2077 - f1_score: 0.1703 - loss: 2.0915 - precision: 0.4286 - recall: 0.0017 - val_accuracy: 0.2870 - val_f1_score: 0.2069 - val_lo

In [15]:
results = model.evaluate(test_generator)
print(f"Test Loss: {results[0]}")
print(f"Test Accuracy: {results[1]}")

print(results)


predictions = model.predict(test_generator)
# Get predicted classes directly from predictions
y_pred = np.argmax(predictions, axis=1)
# Get true class labels
y_true = test_generator.classes

print("predicted values : ",y_pred)
print("True values      : ",y_true)


print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred))



predictions = model.predict(test_generator)
# Get predicted classes directly from predictions
predicted_classes = np.argmax(predictions, axis=1)
# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 563ms/step - accuracy: 0.4912 - f1_score: 0.2957 - loss: 3.1344 - precision: 0.5930 - recall: 0.4279
Test Loss: 3.0327987670898438
Test Accuracy: 0.4444444477558136
[3.0327987670898438, 0.4444444477558136, 0.5359477400779724, 0.37962964177131653, <tf.Tensor: shape=(9,), dtype=float32, numpy=
array([0.88461536, 0.38095233, 0.490566  , 0.04878044, 0.40909082,
       0.05882349, 0.35483864, 0.2222222 , 0.96      ], dtype=float32)>]
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step
predicted values :  [0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 4 4 7 6 1 2 1 1 1 1 3 1
 1 6 6 1 7 7 6 6 6 3 7 7 2 3 3 5 3 2 2 2 2 3 2 5 7 3 2 2 2 2 2 2 5 2 6 7 4
 0 8 1 1 1 1 0 2 7 7 7 6 6 7 7 5 7 1 6 3 4 6 4 5 4 6 6 6 7 3 6 6 7 4 6 4 6
 6 4 4 4 7 6 4 4 6 7 4 0 8 1 1 1 1 0 2 7 7 7 6 6 7 7 5 7 1 6 3 4 6 6 6 7 6
 6 6 6 6 2 4 4 2 4 3 6 6 6 5 2 2 2 2 2 6 5 5 2 7 2 7 7 7 0 7 5 3 2 7 2 3 6
 6 3 2 3 3 3 2 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
True values      :  [0 0 0 0 0 0 0 0 0 0 0 